In [11]:
import os
import pandas as pd
import numpy as np

In [2]:
os.getcwd()

'/Users/vedrai/Downloads/output'

In [4]:
#Gets the list of all preds labels
preds_labels_txts = os.listdir(os.path.join(os.getcwd(), 'preds_labels'))

In [6]:
#Gets the list of all true labels
true_labels_txts = os.listdir(os.path.join(os.getcwd(), 'true_labels'))

In [45]:
#Creates empty dataframe
complete_preds = pd.DataFrame(columns=['label', 'x1', 'y1','x2','y2', 'fname'])
complete_true = pd.DataFrame(columns=['label', 'x1', 'y1','x2','y2', 'fname'])

In [46]:
#Stacks in a pd dataframe all the annots true
for file in true_labels_txts:
    true_instance = pd.read_csv(f'true_labels/{file}', names=['label', 'x1', 'y1','x2','y2'], delim_whitespace=True, index_col=False)
    true_instance['fname'] = file
    complete_true = pd.concat((complete_true, true_instance))

In [47]:
#Stacks in a dataframe all the annotations predictions
for file in preds_labels_txts:
    pred_instance = pd.read_csv(f'preds_labels/{file}', names=['label', 'x1', 'y1','x2','y2'], delim_whitespace=True, index_col=False)
    pred_instance['fname'] = file
    complete_preds = pd.concat((complete_preds, pred_instance))

In [53]:
#Merges the files on file name
merged_df = pd.merge(complete_preds,complete_true, on='fname')

In [56]:
#Takes where the predicted class is the same as the true class
filtered_merged_df = merged_df[merged_df['label_x']==merged_df['label_y']]

In [59]:
#Just resetting the index
filtered_merged_df.reset_index(drop=True,inplace=True)

In [79]:
#Amongst those for each computes the IoU and takes the one which is between 0 and 1
#Meaning the best among the existing ones, if they're all shit it's going to take shit
for row in filtered_merged_df.iterrows():
    true_box = row[1][['x1_x', 'y1_x', 'x2_x', 'y2_x']]
    pred_box = row[1][['x1_y', 'y1_y', 'x2_y', 'y2_y']]
    filtered_merged_df.loc[row[0], 'IoU'] = bb_intersection_over_union(true_box, pred_box)

/Users/vedrai/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [95]:
filtered_merged_df

,label_x,x1_x,y1_x,x2_x,y2_x,fname,label_y,x1_y,y1_y,x2_y,y2_y,IoU
0,11,0.350781,0.557813,0.273438,0.215625,houston_1538054647864322395_20170615_jpg.rf.ed...,11,0.350781,0.558594,0.268750,0.210938,0.986648
1,11,0.550781,0.605469,0.217187,0.220313,houston_1796595615124218317_20180607_jpg.rf.dd...,11,0.548438,0.606250,0.221094,0.215625,0.981977
2,6,0.169531,0.930469,0.123438,0.139062,miami_1453941252022583994_20170219_jpg.rf.59ef...,6,0.168750,0.927344,0.125781,0.142187,0.967736
3,10,0.540625,0.369531,0.068750,0.067187,la_2017_1641095634084003690_20171105_jpg.rf.e8...,10,0.543750,0.369531,0.065625,0.070312,0.983811
4,8,0.872656,0.082812,0.157813,0.059375,la_1318265683658513474_20160816_jpg.rf.9f10cdc...,8,0.871875,0.082812,0.156250,0.059375,0.991802
...,...,...,...,...,...,...,...,...,...,...,...,...
2356,0,0.222656,0.732031,0.085938,0.060937,la_1244405538751206374_20160506_jpg.rf.137acd2...,0,0.221875,0.733594,0.088281,0.076563,0.946702
2357,8,0.871094,0.082031,0.157813,0.064062,chicago_1071175752464303275_20150910_jpg.rf.e3...,8,0.871875,0.082812,0.158594,0.070312,0.987511
2358,1,0.676562,0.366406,0.062500,0.079687,houston_1570329889772864611_20170730_jpg.rf.8e...,1,0.675781,0.366406,0.065625,0.079687,0.989981
2359,1,0.186719,0.757812,0.085938,0.050000,la_1203737777393789214_20160311_jpg.rf.7394a3f...,1,0.184375,0.760156,0.083594,0.059375,0.956237


In [85]:
#Here we group by the files by max IoU for each (since we have many possible predictions we get the one
#being the closest to the real bounding box, which we will consider as the valid one)
final_df = filtered_merged_df.groupby('fname').max('IoU').reset_index(inplace=True)

In [93]:
#Simple index resetting
final_df.reset_index(inplace=True)

In [101]:
#Put back the original classifications
dff = pd.merge(final_df, complete_preds[['label', 'fname']], on='fname')

In [104]:
#Group by to show average IoU
dff.groupby('label').mean('IoU')['IoU']

label
0     0.972089
1     0.969230
2     0.939702
3     0.954608
5     0.959567
6     0.976084
7     0.965227
8     0.966581
9     0.944344
10    0.960205
11    0.969744
12    0.959215
13    0.960630
14    0.968717
Name: IoU, dtype: float64

In [25]:
#True code for computing IoU in [0,1]
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth===
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [26]:
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_img_bbox(img, target, target2):
  fig, a = plt.subplots(1,1)
  fig.set_size_inches(5,5)
  a.imshow(img)
  
  # predicted box
  x, y, width, height  = target[0], target[1], target[2], target[3]
  rect = patches.Rectangle(
    (x, y),
    width, height,
    linewidth = 2,
    edgecolor = 'r',
    facecolor = 'none'
  )
  # true box 
  x, y, width, height  = target2[0], target2[1], target2[2], target2[3]
  a.add_patch(rect)
  rect = patches.Rectangle(
    (x, y),
    width, height,
    linewidth = 2,
    edgecolor = 'green',
    facecolor = 'none'
  )
  a.add_patch(rect)
  plt.show()

def xywhn2xyxy(x, w=640, h=640, padw=0, padh=0):
    # Convert nx4 boxes from [x, y, w, h] normalized to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = np.copy(x)
    y[0] = w * (x[0] - x[2] / 2) + padw  # top left x
    y[1] = h * (x[1] - x[3] / 2) + padh  # top left y
    y[2] = w * (x[0] + x[2] / 2) + padw  # bottom right x
    y[3] = h * (x[1] + x[3] / 2) + padh  # bottom right y
    return y

In [49]:
import pandas as pd
import numpy as np

preds_path = '/Users/aliyadavletshina/Downloads/all_labels/preds_labels'
true_path = '/Users/aliyadavletshina/Downloads/all_labels/true_labels'
image_names = os.listdir(preds_path)
pics = '/Users/aliyadavletshina/Desktop/Bocconi/DL-CV/logo-detection-local/raw_data/train'


predictions = pd.DataFrame(columns=['image_name', 'true_class', 'x1t', 'y1t', 'wt', 'ht',
                                                    'pred_class', 'x1', 'y1', 'w', 'h', 
                                                    'conf', 'width', 'height', 'iou'])

for pic in os.listdir(preds_path):
    if not os.path.exists(os.path.join(pics, pic.split('_jpg')[0]+'.jpg')):
        print(pic)
    im = cv2.imread(os.path.join(pics, pic.split('_jpg')[0]+'.jpg'))
    h, w, _ = im.shape
    with open(os.path.join(preds_path, pic), 'r') as f:
        preds = f.readlines()
        # .split(' ')
        f.close()
        # preds = [float(i) for i in preds]
    with open(os.path.join(true_path, pic), 'r') as f:
        truelbs = f.readline().split(' ')
        f.close()
        truelbs = [float(i) for i in truelbs]
        for pred in preds:
            pred = pred.split(' ')
            pred = [float(i) for i in pred]

            im = cv2.imread(os.path.join(pics, pic.split('_jpg')[0]+'.jpg'))
            h, w, _ = im.shape

            xyxy_pred = xywhn2xyxy(np.array(pred), w, h, padw=0, padh=0)
            xyxy_true = xywhn2xyxy(np.array(truelbs), w, h, padw=0, padh=0)
            iou = bb_intersection_over_union(xyxy_pred, xyxy_true)

            predictions = predictions.append({'image_name': pic, 'true_class': int(truelbs[0]),
                                    'x1t': truelbs[1], 'y1t': truelbs[2], 'wt':truelbs[3], 'ht':truelbs[4],
                                    'pred_class': int(pred[0]), 'x1': float(pred[1]),
                                    'y1': float(pred[2]), 'w':float(pred[3]), 'h': float(pred[4]),
                                    'conf':float(pred[5]), 'width': w, 'height': h, 'iou': iou}, ignore_index=True)

    # plotting boxes
    # im = cv2.imread(os.path.join(pics, pic.split('_jpg')[0]+'.jpg'))
    # h, w, _ = im.shape

    # xyxy_pred = xywhn2xyxy(np.array(preds), w, h, padw=0, padh=0)
    # xyxy_true = xywhn2xyxy(np.array(truelbs), w, h, padw=0, padh=0)

    # true_box = [float(i) for i in [truelbs[1], truelbs[2], truelbs[3],truelbs[4]]]
    # pred_box = [float(i) for i in [preds[1], preds[2], preds[3], preds[4]]]
    # true_box = [(true_box[0]-true_box[2])*w, (true_box[1]-true_box[3])*h, true_box[2]*w, true_box[3]*h]
    # pred_box = [(pred_box[0]-pred_box[2])*w, (pred_box[1]-pred_box[3])*h, pred_box[2]*w, pred_box[3]*h]
    # print(f'true box {true_box}\n pred box {pred_box}')
    # plot_img_bbox(im, true_box, pred_box)

    # print(bb_intersection_over_union(xyxy_pred, xyxy_true))


In [56]:
predictions_iou = predictions.copy()

for idx, row in predictions_iou.iterrows():
    if row['true_class'] != row['pred_class']:
        row['iou'] = 0
idx = predictions_iou.groupby('image_name')['conf'].transform(max) == predictions_iou['conf']
predictions_iou = predictions_iou[idx]

predictions_iou.groupby('true_class')['iou'].mean()


true_class
0     0.929157
1     0.925126
2     0.874753
3     0.897359
4     0.000000
5     0.905461
6     0.947197
7     0.926856
8     0.925456
9     0.793263
10    0.902524
11    0.950983
12    0.936483
13    0.853026
14    0.928360
Name: iou, dtype: float64

In [60]:
predictions_iou[predictions_iou['true_class'] == 4]

,image_name,true_class,x1t,y1t,wt,ht,pred_class,x1,y1,w,h,conf,width,height,iou
1085,chicago_1302469769613898571_20160725_jpg.rf.d3...,4,0.647656,0.775,0.204687,0.114062,0,0.189844,0.449219,0.048438,0.045312,0.903889,1000,1000,0.0


In [66]:
predictions_iou.groupby('true_class')['true_class'].count()

true_class
0     451
1     151
2      39
3      90
4       1
5      66
6     193
7     142
8     320
9      50
10     69
11    340
12     93
13     68
14    120
Name: true_class, dtype: int64